In [2]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import pandas as pd
import csv

Simple linear/logistic regression would be my first go to choice in this scenario, but an SVM could work good as well.

In [3]:
base_train = pd.read_csv('train.csv')
base_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
base_train.loc[base_train['Sex'] == 'male', 'Sex'] = 1
base_train.loc[base_train['Sex'] == 'female', 'Sex'] = 0
base_train.loc[base_train['Embarked'] == 'S', 'Embarked'] = 1
base_train.loc[base_train['Embarked'] == 'C', 'Embarked'] = 2
base_train.loc[base_train['Embarked'] == 'Q', 'Embarked'] = 3
# base_train['Embarked'].unique()
base_train[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,-0.338481,-0.543351,-0.077221,-0.035322,0.081629,0.257307,0.108669
Pclass,-0.338481,1.000000,0.131900,-0.369226,0.083081,0.018443,-0.549500,0.043835
Sex,-0.543351,0.131900,1.000000,0.093254,-0.114631,-0.245489,-0.182333,-0.118593
Age,-0.077221,-0.369226,0.093254,1.000000,-0.308247,-0.189119,0.096067,0.012186
SibSp,-0.035322,0.083081,-0.114631,-0.308247,1.000000,0.414838,0.159651,-0.060606
Parch,0.081629,0.018443,-0.245489,-0.189119,0.414838,1.000000,0.216225,-0.079320
Fare,0.257307,-0.549500,-0.182333,0.096067,0.159651,0.216225,1.000000,0.063462
Embarked,0.108669,0.043835,-0.118593,0.012186,-0.060606,-0.079320,0.063462,1.000000


In [5]:
target = base_train['Survived'].values.reshape(-1,1)
# Replace nan values with 0
base_train['Age'] = base_train['Age'].fillna(0)
base_train['Embarked'] = base_train['Embarked'].fillna(0)
features = base_train[['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked']].values
features

C:\Users\Skidro\AppData\Local\Temp\ipykernel_30012\2895307094.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  base_train['Embarked'] = base_train['Embarked'].fillna(0)


array([[3, 1, 22.0, ..., 0, 7.25, 1],
       [1, 0, 38.0, ..., 0, 71.2833, 2],
       [3, 0, 26.0, ..., 0, 7.925, 1],
       ...,
       [3, 0, 0.0, ..., 2, 23.45, 1],
       [1, 1, 26.0, ..., 0, 30.0, 2],
       [3, 1, 32.0, ..., 0, 7.75, 3]], dtype=object)

In [6]:
regressor = LogisticRegression()
regressor.fit(features,target)
print(regressor.intercept_)

test = pd.read_csv('test.csv')
test.loc[test['Sex'] == 'male', 'Sex'] = 1
test.loc[test['Sex'] == 'female', 'Sex'] = 0
test.loc[test['Embarked'] == 'S', 'Embarked'] = 1
test.loc[test['Embarked'] == 'C', 'Embarked'] = 2
test.loc[test['Embarked'] == 'Q', 'Embarked'] = 3
test['Age'] = test['Age'].fillna(0)
test['Fare'] = test['Fare'].fillna(0)
test_features = test[['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked']]

output = regressor.predict(test_features)
with open('output.csv', 'w', newline='') as csv_file:
    file_writer = csv.writer(csv_file, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    file_writer.writerow(['PassengerId','Survived'])
    ids = test['PassengerId'].values
    for i in range(len(ids)):
        file_writer.writerow([ids[i], output[i]])

# Most Basic one has an output of 76.7% accuracy

[3.5570471]


c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Trying to use the mean and median imputed datasets now:



In [7]:
mean_train = pd.read_csv('mean_imputed_data.csv')
mean_train.loc[mean_train['Sex'] == 'male', 'Sex'] = 1
mean_train.loc[mean_train['Sex'] == 'female', 'Sex'] = 0
mean_train.loc[mean_train['Embarked'] == 'S', 'Embarked'] = 1
mean_train.loc[mean_train['Embarked'] == 'C', 'Embarked'] = 2
mean_train.loc[mean_train['Embarked'] == 'Q', 'Embarked'] = 3
mean_train['Embarked'] = mean_train['Embarked'].fillna(0)
features = mean_train[['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked']].values
mean_regressor = LogisticRegression()
mean_regressor.fit(features,target)

output = mean_regressor.predict(test_features)
with open('mean_output.csv', 'w', newline='') as csv_file:
    file_writer = csv.writer(csv_file, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    file_writer.writerow(['PassengerId','Survived'])
    ids = test['PassengerId'].values
    for i in range(len(ids)):
        file_writer.writerow([ids[i], output[i]])

# Mean imputation really didnt help much in this case as we ended up with a accuracy of 75.19%

C:\Users\Skidro\AppData\Local\Temp\ipykernel_30012\1581841907.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mean_train['Embarked'] = mean_train['Embarked'].fillna(0)
c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.h

In [8]:
median_train = pd.read_csv('median_imputed_data.csv')
median_train.loc[median_train['Sex'] == 'male', 'Sex'] = 1
median_train.loc[median_train['Sex'] == 'female', 'Sex'] = 0
median_train.loc[median_train['Embarked'] == 'S', 'Embarked'] = 1
median_train.loc[median_train['Embarked'] == 'C', 'Embarked'] = 2
median_train.loc[median_train['Embarked'] == 'Q', 'Embarked'] = 3
median_train['Embarked'] = median_train['Embarked'].fillna(0)
features = median_train[['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked']].values
median_regressor = LogisticRegression()
median_regressor.fit(features,target)

output = median_regressor.predict(test_features)
with open('median_output.csv', 'w', newline='') as csv_file:
    file_writer = csv.writer(csv_file, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    file_writer.writerow(['PassengerId','Survived'])
    ids = test['PassengerId'].values
    for i in range(len(ids)):
        file_writer.writerow([ids[i], output[i]])

# Median imputation really didnt help much in this case as we ended up with a accuracy of 75.19%

C:\Users\Skidro\AppData\Local\Temp\ipykernel_30012\3065453342.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  median_train['Embarked'] = median_train['Embarked'].fillna(0)
c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessi

Next we will try to extract the first letter of the Cabin, encode it and see if that makes much of a difference.


In [9]:
def select_first(cabin):
    if cabin:
        if str(cabin)[0] != 'n':
            return ord(str(cabin)[0])
# base_train['Cabin']
base_train['Cabin'] = base_train['Cabin'].apply(select_first)
base_train['Cabin'] = base_train['Cabin'].fillna(0)


In [12]:
base_train.info()
features = base_train[['Pclass','Sex','Age','SibSp','Parch','Fare', 'Cabin', 'Embarked']].values
base_regression = LogisticRegression()
base_regression.fit(features, target)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    float64
 11  Embarked     891 non-null    int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 83.7+ KB


c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
test_features = test[['Pclass','Sex','Age','SibSp','Parch','Fare', 'Cabin' ,'Embarked']]
def select_first(cabin):
    if cabin:
        if str(cabin)[0] != 'n':
            return ord(str(cabin)[0])
# base_train['Cabin']
test_features['Cabin'] = test_features['Cabin'].apply(select_first)
test_features['Cabin'] = test_features['Cabin'].fillna(0)

C:\Users\Skidro\AppData\Local\Temp\ipykernel_30012\3410859811.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_features['Cabin'] = test_features['Cabin'].apply(select_first)
C:\Users\Skidro\AppData\Local\Temp\ipykernel_30012\3410859811.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_features['Cabin'] = test_features['Cabin'].fillna(0)


In [16]:
output = base_regression.predict(test_features)
with open('cabin_copy_output.csv', 'w', newline='') as csv_file:
    file_writer = csv.writer(csv_file, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    file_writer.writerow(['PassengerId','Survived'])
    ids = test['PassengerId'].values
    for i in range(len(ids)):
        file_writer.writerow([ids[i], output[i]])

c:\Users\Skidro\.conda\envs\pytorch\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
